<a href="https://colab.research.google.com/github/CheilaBaiao/GEE_SR/blob/main/Pantanal_pre_processamento_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#(Prepara o ambiente, autentica no EE e monta o Drive.)
!pip -q install earthengine-api geopandas shapely

from google.colab import drive
drive.mount('/content/drive')

import ee, datetime, json, sys, geopandas as gpd
import shapely.geometry as sgeom
from shapely.geometry import mapping

try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize(project="ee-cheilabaiao")

print("EE OK às", datetime.datetime.now())


Mounted at /content/drive
EE OK às 2025-09-09 23:18:53.539552


In [2]:
#(Define anos, resolução/CRS, sentinel NoData e pastas.)
# ===== PARÂMETROS =====
PROJECT        = "ee-cheilabaiao"            # seu projeto EE (opcional no Initialize)
DRIVE_FOLDER   = "Pantanal_TippingPoints"     # pasta no Google Drive
PREFIX         = "NDVI_Pantanal"              # prefixo dos arquivos
START_YEAR     = 1985
END_YEAR       = 2024
CRS            = "EPSG:3857"                  # projeção em METROS
EXPORT_SCALE   = 300                           # 300 m
NODATA         = -32768                        # sentinel seguro para Int16 ×10000

proj300 = ee.Projection(CRS).atScale(EXPORT_SCALE)

print(json.dumps({
    "folder": DRIVE_FOLDER,
    "prefix": PREFIX,
    "years": [START_YEAR, END_YEAR],
    "crs": CRS,
    "scale_m": EXPORT_SCALE,
    "nodata": NODATA
}, indent=2))

{
  "folder": "Pantanal_TippingPoints",
  "prefix": "NDVI_Pantanal",
  "years": [
    1985,
    2024
  ],
  "crs": "EPSG:3857",
  "scale_m": 300,
  "nodata": -32768
}


In [3]:
#(Lê o shapefile, corrige geometrias, dissolve no EE; cria roi_export (bbox) e mask300.)
# ===== SHAPEFILE → pantanal =====
SHP_PATH = '/content/drive/MyDrive/Pantanal/Pantanal.shp'  # <<< ajuste

# Força 2D (sem Z/M) compatível com várias versões do Shapely
def force_2d_any(geom):
    try:
        from shapely import force_2d  # shapely >= 2.0
        return force_2d(geom)
    except Exception:
        if geom.is_empty: return geom
        if isinstance(geom, sgeom.Point): return sgeom.Point(geom.x, geom.y)
        if isinstance(geom, sgeom.LineString): return sgeom.LineString([(x,y) for x,y,*_ in geom.coords])
        if isinstance(geom, sgeom.LinearRing): return sgeom.LinearRing([(x,y) for x,y,*_ in geom.coords])
        if isinstance(geom, sgeom.Polygon):
            ext = [(x,y) for x,y,*_ in geom.exterior.coords]
            ints = [[(x,y) for x,y,*_ in r.coords] for r in geom.interiors]
            return sgeom.Polygon(ext, ints)
        if isinstance(geom, sgeom.MultiPolygon):
            return sgeom.MultiPolygon([force_2d_any(g) for g in geom.geoms])
        if isinstance(geom, sgeom.GeometryCollection):
            return sgeom.GeometryCollection([force_2d_any(g) for g in geom.geoms])
        return geom

gdf = gpd.read_file(SHP_PATH)
if gdf.crs is None or gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)
try:
    gdf = gdf.explode(index_parts=False)
except TypeError:
    gdf = gdf.explode()

gdf['geometry'] = gdf['geometry'].apply(force_2d_any).buffer(0)
gdf = gdf[gdf.is_valid]

# Dissolve server-side (evita RecursionError e alto uso de memória)
features = [ee.Feature(ee.Geometry(mapping(geom), None, False, True)) for geom in gdf.geometry]
pantanal = ee.FeatureCollection(features).geometry()

# Região simples (bbox) em CRS alvo + máscara raster 300 m (1 dentro; fora = máscara)
roi_export = ee.Geometry(pantanal).transform(CRS, 1).bounds(maxError=100)
mask300 = ee.Image.constant(1).toByte().reproject(proj300).clip(pantanal)

# Diagnóstico rápido de área (pixelArea na mesma grade)
area_m2 = (ee.Image.pixelArea().reproject(proj300).updateMask(mask300)
           .reduceRegion(ee.Reducer.sum(), geometry=roi_export, crs=CRS,
                         scale=EXPORT_SCALE, bestEffort=True, tileScale=16, maxPixels=1e13)
           .get('area').getInfo())
print('Área (km²) ~', area_m2/1e6)


Área (km²) ~ 151806.60001063062


In [4]:
# 4) (Converte SR para reflectância, define RED/NIR por sensor e calcula NDVI com normalizedDifference.)
# ===== FUNÇÕES LANDSAT L2 =====
def scale_sr(img):
    sr = img.select('SR_B.*').multiply(0.0000275).add(-0.2)   # SR→reflectância
    return img.addBands(sr, overwrite=True)

def add_red_nir(img, sensor):
    if sensor == 'L89':   # Landsat 8/9
        red = img.select('SR_B4'); nir = img.select('SR_B5')
    else:                 # Landsat 5/7
        red = img.select('SR_B3'); nir = img.select('SR_B4')
    return img.addBands([red.rename('RED'), nir.rename('NIR')])

def add_ndvi(img):
    return img.addBands(img.normalizedDifference(['NIR','RED']).rename('NDVI'))

# ===== COLEÇÕES NDVI =====
L5 = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
      .filterBounds(pantanal).filterDate('1985-01-01', '2013-07-01')
      .map(scale_sr).map(lambda im: add_red_nir(im,'L57')).map(add_ndvi)
      .select(['NDVI']))

L7 = (ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
      .filterBounds(pantanal).filterDate('1999-01-01', '2024-12-31')
      .map(scale_sr).map(lambda im: add_red_nir(im,'L57')).map(add_ndvi)
      .select(['NDVI']))

L8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
      .filterBounds(pantanal).filterDate('2013-04-11', '2024-12-31')
      .map(scale_sr).map(lambda im: add_red_nir(im,'L89')).map(add_ndvi)
      .select(['NDVI']))

L9 = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
      .filterBounds(pantanal).filterDate('2021-11-01', '2024-12-31')
      .map(scale_sr).map(lambda im: add_red_nir(im,'L89')).map(add_ndvi)
      .select(['NDVI']))

LS_NDVI = L5.merge(L7).merge(L8).merge(L9)
print('Cenas NDVI na coleção total:', LS_NDVI.size().getInfo())


Cenas NDVI na coleção total: 17618


In [5]:
# 5) (Por cena: 30→300 m (média) dentro da bbox; depois mediana temporal a 300 m; aplica mask300 e sentinel.)
# ===== NDVI mensal (300 m) e stack anual =====
def monthly_ndvi_300m(year: int, month: int) -> ee.Image:
    ini = ee.Date.fromYMD(year, month, 1); end = ini.advance(1, 'month')
    coll = LS_NDVI.filterDate(ini, end).filterBounds(roi_export)
    nscenes = coll.size()

    # Por cena: recorta à bbox e agrega 30 m → 300 m (média); define grade fixa
    def _to300(im):
        return (im.select('NDVI')
                  .clip(roi_export)
                  .reduceResolution(ee.Reducer.mean(), maxPixels=65535)
                  .setDefaultProjection(proj300))

    coll300 = coll.map(_to300)

    # Mediana temporal já em 300 m; aplica máscara raster do Pantanal
    nd300 = ee.Image(ee.Algorithms.If(nscenes.gt(0), coll300.median(), ee.Image(0).updateMask(ee.Image(0))))
    nd300 = nd300.updateMask(mask300)

    im = (nd300.multiply(10000).round().toInt16()
               .unmask(NODATA)
               .rename(f'NDVI_{month:02d}')
               .set({'system:time_start': ini.millis(), 'count': nscenes}))
    return im

def yearly_stack_300m(year: int) -> ee.Image:
    bands = [monthly_ndvi_300m(year, m) for m in range(1,13)]
    return (ee.Image.cat(bands)
            .clip(roi_export)  # bbox simples (barato)
            .set({'year': year, 'system:time_start': ee.Date.fromYMD(year,1,1).millis()}))


In [7]:
# (Usa a mesma grade/região do export; ignora sentinel na estatística.)
# ===== QA pré-export: percentis mensais (mesma grade/região do export) =====
import pandas as pd

def monthly_percentiles_300m(year: int, perc=[5,25,50,75,95]) -> pd.DataFrame:
    rows=[]
    for m in range(1,13):
        im = monthly_ndvi_300m(year, m)
        im_valid = im.updateMask(im.neq(NODATA))
        s = (im_valid.reduceRegion(
                reducer=ee.Reducer.percentile(perc),
                geometry=roi_export, crs=CRS, scale=EXPORT_SCALE,
                bestEffort=True, tileScale=16, maxPixels=1e13)
             .getInfo() or {})
        row={'band': f'NDVI_{m:02d}'}
        for p in perc:
            key=f'NDVI_{m:02d}_p{p}'
            val = s.get(key)
            row[f'p{p}'] = (val/10000.0) if val is not None else None
        rows.append(row)
    return pd.DataFrame(rows)

TEST_YEAR = 2004
df_pre = monthly_percentiles_300m(TEST_YEAR)
df_pre


,band,p5,p25,p50,p75,p95
0,NDVI_01,0.156759,0.361572,0.495955,0.611135,0.739140
1,NDVI_02,-0.393695,0.060773,0.201560,0.393531,0.636749
2,NDVI_03,0.201543,0.470367,0.579140,0.668758,0.790313
3,NDVI_04,0.284763,0.495981,0.585570,0.675141,0.796699
4,NDVI_05,0.182393,0.399954,0.553563,0.649530,0.783928
5,NDVI_06,0.278337,0.508793,0.591965,0.675170,0.783932
6,NDVI_07,0.297568,0.451179,0.534360,0.623945,0.732702
7,NDVI_08,0.323195,0.412775,0.476736,0.566341,0.700722
8,NDVI_09,0.150379,0.278375,0.342351,0.406322,0.547113
9,NDVI_10,-0.099202,0.105588,0.233565,0.342326,0.495887


In [8]:
# 7) (Exporta um ano como stack de 12 bandas, com noData e bbox simples; inclui monitor simples das tasks.)
# ===== Export anual @ 300 m (COG) =====
def export_one_year(year: int):
    img = yearly_stack_300m(year)
    desc = f'{PREFIX}_{year}_stack_300m'
    task = ee.batch.Export.image.toDrive(
        image=img,
        description=desc,
        folder=DRIVE_FOLDER,
        fileNamePrefix=desc,
        region=roi_export,              # bbox simples
        crs=CRS,
        scale=EXPORT_SCALE,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True, 'noData': NODATA}
    )
    task.start()
    print('> Export iniciado:', desc)

# Exemplo: exporta o ano de teste
export_one_year(TEST_YEAR)

# Monitor
for t in ee.batch.Task.list():
    st = t.status()
    print('-', st.get('description'), '|', st.get('state'))


> Export iniciado: NDVI_Pantanal_2004_stack_300m
- NDVI_Pantanal_2004_stack_300m | READY
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | CANCELLED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED
- NDVI_Pantanal_2005_stack_300m | COMPLETED


In [ ]:
# 8) (Conta cenas por mês em um ano para entender vazios.)

def months_check(y):
    empty=[]; counts=[]
    for m in range(1,13):
        c = (LS_NDVI.filterBounds(roi_export)
                     .filterDate(ee.Date.fromYMD(y,m,1), ee.Date.fromYMD(y,m,1).advance(1,'month'))
                     .size().getInfo())
        counts.append(int(c))
        if c==0: empty.append(m)
    print(f'{y}: cenas/mês =', counts, '| meses vazios:', empty or 'nenhum')

months_check(TEST_YEAR)


In [14]:
# 9) (Confere dtype/NoData/COG e percentis; devem bater com a célula 6, salvo diferenças numéricas pequenas.)
!pip -q install rasterio numpy

import rasterio, numpy as np, os, math, pandas as pd

tif_path = "/content/drive/MyDrive/Pantanal_TippingPoints/NDVI_Pantanal_2004_stack_300m.tif"

with rasterio.open(tif_path) as ds:
    print("Arquivo:", os.path.basename(tif_path))
    print("Bandas:", ds.count)
    print("CRS   :", ds.crs)
    print("Dtype :", ds.dtypes[0])
    print("NoData:", ds.nodatavals[0])
    print("Tiled :", ds.profile.get("tiled", None))
    print("Overviews b1:", ds.overviews(1))
    print("Transform:", ds.transform)
    if ds.crs and ds.crs.is_projected:
        print(f"Pixel ~ {abs(ds.transform.a):.2f} m × {abs(ds.transform.e):.2f} m")

    scale = 10000.0 if ds.dtypes[0]=='int16' else 1.0
    H, W = ds.height, ds.width
    stats=[]
    for b in range(1, ds.count+1):
        arr = ds.read(b, masked=True).astype('float32')
        vals = (arr.compressed()/scale) if arr.mask is not None else (arr/scale).ravel()
        if vals.size==0:
            stats.append({'band': f'NDVI_{b:02d}','p5':None,'p25':None,'p50':None,'p75':None,'p95':None})
        else:
            stats.append({
                'band': f'NDVI_{b:02d}',
                'p5': float(np.percentile(vals,5)),
                'p25': float(np.percentile(vals,25)),
                'p50': float(np.percentile(vals,50)),
                'p75': float(np.percentile(vals,75)),
                'p95': float(np.percentile(vals,95)),
            })

df_post = pd.DataFrame(stats)
display(df_post)
print("\nDiferenças |pós - pré| (esperado ~0):")
df_cmp = df_post.merge(df_pre, on='band', suffixes=('_post','_pre'))
for p in [5,25,50,75,95]:
    df_cmp[f'd_p{p}'] = (df_cmp[f'p{p}_post'] - df_cmp[f'p{p}_pre']).abs()
display(df_cmp[[f'd_p{p}' for p in [5,25,50,75,95]]].round(4))


Arquivo: NDVI_Pantanal_2004_stack_300m.tif
Bandas: 12
CRS   : EPSG:3857
Dtype : int16
NoData: -32768.0
Tiled : True
Overviews b1: [2, 4, 8, 16, 32]
Transform: | 300.00, 0.00,-6583800.00|
| 0.00,-300.00,-1763700.00|
| 0.00, 0.00, 1.00|
Pixel ~ 300.00 m × 300.00 m


,band,p5,p25,p50,p75,p95
0,NDVI_01,0.1565,0.3641,0.4958,0.6083,0.7407
1,NDVI_02,-0.3942,0.0602,0.2009,0.3950,0.6387
2,NDVI_03,0.1998,0.4733,0.5763,0.6668,0.7874
3,NDVI_04,0.2818,0.4928,0.5866,0.6754,0.7965
4,NDVI_05,0.1821,0.4008,0.5509,0.6510,0.7827
5,NDVI_06,0.2802,0.5102,0.5891,0.6755,0.7833
6,NDVI_07,0.2973,0.4532,0.5350,0.6263,0.7332
7,NDVI_08,0.3251,0.4120,0.4761,0.5653,0.6979
8,NDVI_09,0.1534,0.2773,0.3397,0.4095,0.5491
9,NDVI_10,-0.0999,0.1084,0.2311,0.3409,0.4930



Diferenças |pós - pré| (esperado ~0):


,d_p5,d_p25,d_p50,d_p75,d_p95
0,0.0003,0.0025,0.0002,0.0028,0.0016
1,0.0005,0.0006,0.0007,0.0015,0.0020
2,0.0017,0.0029,0.0028,0.0020,0.0029
3,0.0030,0.0032,0.0010,0.0003,0.0002
4,0.0003,0.0008,0.0027,0.0015,0.0012
5,0.0019,0.0014,0.0029,0.0003,0.0006
6,0.0003,0.0020,0.0006,0.0024,0.0005
7,0.0019,0.0008,0.0006,0.0010,0.0028
8,0.0030,0.0011,0.0027,0.0032,0.0020
9,0.0007,0.0028,0.0025,0.0014,0.0029
